### 실습 개요
- 이미지에서 얼굴을 찾습니다.
- 얼굴 이미지들간의 유사한 정도를 측정합니다.
- 이미지에서 특정인의 얼굴을 찾아 다른 사람의 얼굴로 바꿉니다.

### 사전준비
#### 1. 필수 Library들을 설치
##### 1-1. 필수 Library 목록
- **NumPy**: 과학적 계산을 위한 파이썬 라이브러리. 고성능 다차원 배열 객체와 다양한 수학적 함수들을 제공하며, 데이터 분석, 머신 러닝, 신호 처리 등에서 널리 사용됨.

- **OpenCV**: 이미지와 비디오 분석을 위한 컴퓨터 비전 라이브러리. 얼굴 검출, 객체 추적, 이미지 처리에 널리 사용됨.

- **InsightFace**: 얼굴 인식 및 검출을 위한 오픈 소스 라이브러리. ArcFace 기반의 높은 정확도로 보안 및 인증 시스템에 활용됨.

- **ONNX Runtime**: ONNX 형식의 딥러닝 모델을 빠르고 효율적으로 실행하는 런타임.

##### 1-2. 필수 Library 설치
- `Numpy`와 `OpenCV`는 `InsightFace` 설치 시 자동 설치됨

In [ ]:
!pip install insightface onnxruntime

##### 1-3. 필수 Library들 Import

In [ ]:
import numpy as np # NumPy
import cv2 # OpenCV
import matplotlib.pyplot as plt # Matplot
import insightface # InsightFace

#### 2. AI 모델 설치
##### 2-1. Facial Anaysis 모델 설치
- buffalo_l download from : 
  
    https://github.com/deepinsight/insightface/releases

- `C:\models\buffalo_l`에 `buffalo_l.zip` 을 unzip 하고 전역변수에 아래와 같이 저장 위치 명시


In [ ]:
BUFFALO_L_PATH = "C:\\"

##### 2-2. Face Swap 모델 설치
- Download `inswapper_128.onnx` :
  
    https://huggingface.co/ezioruan/inswapper_128.onnx/tree/main

- 전역변수에 저장 위치 명시

In [ ]:
INSWAPPER_PATH = r"C:\models\inswapper_128.onnx"

##### 2-3. 모델 설치 결과
```bash
C:\models
├── inswapper_128.onnx
└── buffalo_l
    ├── 1k3d68.onnx
    ├── 2d106det.onnx
    ├── det_10g.onnx
    ├── genderage.onnx
    └── w600k_r50.onnx
```

#### 3. 공부하기
##### 3-1. Yaw, Pitch, Roll
- **Yaw, Pitch, Roll**는 얼굴의 3D 방향을 나타내는 각도 값
  1. **Yaw (요)**: 얼굴이 좌우로 회전할 때의 각도를 나타냅니다.
  2. **Pitch (피치)**: 얼굴이 위나 아래를 볼 때의 각도를 나타냅니다.
  3. **Roll (롤)**: 머리가 한쪽으로 기울어졌을 때의 각도를 나타냅니다.

    ![Link](./faces/yaw_pitch_roll.jpg)

##### 3-2. 중요한 Class 들 불러오기
- **`FaceAnalysis`**: 얼굴 분석을 위한 클래스입니다. 얼굴 감지 모델을 초기화하고, 입력 이미지에서 얼굴을 찾고 다양한 속성(성별, 나이, 얼굴 특징 등)을 추출하는 데 사용됩니다.
- **`Face`**: 얼굴의 세부 정보를 담고 있는 객체입니다. 감지된 얼굴의 경계 상자(`bbox`), 랜드마크(`landmark`), 특징 벡터(`embedding`), 성별, 나이 등 얼굴에 관한 다양한 속성을 포함합니다.

In [ ]:
from insightface.app import FaceAnalysis
from insightface.app.common import Face

### 얼굴 분석 Code 배우기
#### 1. 얼굴 분석 및 분석 정보 출력
##### 1-1. 얼굴 분석 모델 활성화
- 지정된 경로에서 얼굴 분석 모델을 로드하고, CPU 환경에서 실행하도록 준비

In [ ]:
# buffalo Face Analysis 모델을 메모리에 로드하고 AI 기능 초기화
analyzer = FaceAnalysis(name='buffalo_l', root=BUFFALO_L_PATH) 
analyzer.prepare(ctx_id=-1) # CPU Mode 의미

##### 1-2. 이미지 읽기 및 얼굴 검출
- 두 개의 이미지(뉴진스와 트럼프)를 읽은 후, 각 이미지에서 검출된 얼굴 정보를 `faces` 리스트에 저장

In [ ]:
# 뉴진스 이미지 파일 읽기
newjeans_image = cv2.imread("./faces/newJeans.jpg")

# 얼굴 분석 정보 읽어 faces list에 저장
faces = analyzer.get(newjeans_image)

# 트럼프 이미지 파일 읽기
trump_image = cv2.imread("./faces/trump.jpg")

# faces list 마지막 항목에 트럼프 얼굴 정보 추가
faces = faces + analyzer.get(trump_image)

##### 1-3. 검출된 얼굴 정보 출력
-  얼굴에 대해 성별, 나이, 위치, 얼굴 방향, 랜드마크 좌표, 임베딩 벡터의 크기 등을 출력

In [ ]:
# 얼굴 분석 정보 출력
for face in faces:

    print(f"성별 : {face.gender}, 나이 : {face.age}, 영역 좌표 : {face.bbox} , \
        얼굴 방향 : {face.pose} , 얼굴 중요 지점 좌표 shape {face.landmark_2d_106.shape} , \
        얼굴 식별 Vector shape {face.embedding.shape}")

#### 2. 얼굴 바꾸기(스왑) 모델 활용
##### 2-1. 얼굴바꾸기 AI 모델 활성화
- 지정된 경로의 얼굴 바꾸기 모델을 로드

In [ ]:
# Face Swap 모델을 메모리에 로드하고 AI 기능 초기화
swapper = insightface.model_zoo.get_model( INSWAPPER_PATH )

##### 2-2. 얼굴바꾸기 수행
- 리스트의 마지막 항목(트럼프 얼굴)을 기준으로, 뉴진스 이미지 내의 나머지 얼굴들을 순차적으로 트럼프 얼굴로 교체

In [ ]:
# 트럼프 얼굴 분석 정보
trump_face = faces[-1] # faces list의 마지막 항목

# 뉴진스 이미지의 모든 얼굴을 트럼프 얼굴로 교체
for face in faces[:-1]: # faces list의 처음부터 끝에서 두번째까지 (slicing)
    newjeans_image = swapper.get(newjeans_image, face, trump_face)

##### 2-3. 결과 출력
- OpenCV는 BGR 형식이므로, RGB로 변환한 후 Matplotlib를 통해 결과 이미지를 화면에 출력

In [ ]:
# BGR 이미지를 RGB로 변환해서 화면 출력
output_image_rgb = cv2.cvtColor(newjeans_image, cv2.COLOR_BGR2RGB)
plt.imshow(output_image_rgb)

### Code 작성하기 – 데코레이터를 통한 로직 추상화

공통 기능(이미지 읽기, 얼굴 검출, 결과 출력)들을 데코레이터로 추상화하여, 앱 개발자가 저수준의 처리는 신경 쓰지 않고 얼굴에 대한 특정 기능(박스 그리기, 나이 표시, 각도 표시, 랜드마크 표시, 유사도 계산, 얼굴 교체 등)만 구현할 수 있도록 설계

#### 1. 얼굴 분석 및 분석 정보 출력
##### 1-1. 공통 기능 데코레이터: `detect_and_process_faces`

  - 입력 이미지 경로를 받아 이미지를 읽음.  
  - `analyzer.get()`을 사용해 얼굴을 검출한 후, 검출된 각 얼굴에 대해 데코레이터로 감싼 함수(앱 개발자가 구현한 기능)를 호출.
  - 최종 처리된 이미지를 RGB로 변환하여 화면에 출력.

In [ ]:
# buffalo Face Analysis 모델을 메모리에 로드하고 AI 기능 초기화
analyzer = FaceAnalysis(name='buffalo_l', root=BUFFALO_L_PATH) 
analyzer.prepare(ctx_id=-1) # CPU Mode 의미

# Library 개발자 영역 : 데코레이터 정의
def detect_and_process_faces(func):
    
    def wrapper(input_path: str):
        # 입력 이미지 읽기
        image = cv2.imread(input_path)
        if image is None:
            print("이미지를 읽을 수 없습니다. 경로를 확인하세요.")
            return

        # 얼굴 감지 및 분석
        faces = analyzer.get(image)

        # 얼굴에 대해 draw_face_bbox 함수 실행
        for face in faces:
            func(image, face)

        # 결과 이미지 저장
        #cv2.imwrite(output_path, image)
        #print(f"얼굴 영역이 표시된 이미지가 {output_path}에 저장되었습니다.")
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(15, 9))
        plt.imshow(image_rgb)
        plt.axis("off")
        plt.show()

    return wrapper

##### 1-2. 데코레이터 `detect_and_process_faces`를 활용한 다양한 기능 구현

  - 얼굴 영역에 박스 그리기

In [ ]:
# App 개발자 영역 : 얼굴 영역에 박스 그리기
@detect_and_process_faces
def draw_face_bbox(input_image: np.ndarray, face: Face) -> None:
    # 얼굴 영역 표시
    bbox = face.bbox.astype(int)
    cv2.rectangle(input_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)

In [ ]:
# 사용자 영역
draw_face_bbox("./faces/newJeans.jpg")

- 얼굴 옆에 나이 표시

In [ ]:
# App 개발자 영역 : 얼굴 옆에 나이 보여주기
@detect_and_process_faces
def show_age(input_image: np.ndarray, face: Face) -> None:
    # 박스 아래에 나이 정보 표시
    bbox = face.bbox.astype(int)
    cv2.putText(input_image, f"Age: {face.age}", (bbox[0] + 5, bbox[1] + 20), \
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (127, 127, 127), 2)

In [ ]:
# 사용자 영역
show_age("./faces/newJeans.jpg")

- 얼굴 각도(YAW, PITCH, ROLL) 표시

In [ ]:
# App 개발자 영역 : 얼굴 각도 표시하기
@detect_and_process_faces
def draw_ypr(input_image: np.ndarray, face: Face) -> None:
    # 얼굴 영역 표시
    bbox = face.bbox.astype(int)
    cv2.rectangle(input_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    
    # YAW, PITCH, ROLL 계산
    yaw, pitch, roll = face.pose[1], face.pose[0], face.pose[2]

    # YAW, PITCH, ROLL 값 표시
    text_color = (127, 127, 127)
    cv2.putText(input_image, f"YAW: {yaw:.1f}", (bbox[0] + 5, bbox[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
    cv2.putText(input_image, f"PITCH: {pitch:.1f}", (bbox[0] + 5, bbox[1] + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
    cv2.putText(input_image, f"ROLL: {roll:.1f}", (bbox[0] + 5, bbox[1] + 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

In [ ]:
# 사용자 영역
draw_ypr("./faces/newJeans.jpg")

- 얼굴 랜드마크 표시

In [ ]:
# App 개발자 영역 : 얼굴 랜드마크 표시하기
@detect_and_process_faces
def draw_landmarks(input_image: np.ndarray, face: Face) -> None:

    # 각 랜드마크 좌표에 원 그리기
    landmarks = face.landmark_2d_106
    # Draw each landmark point
    for i in range(landmarks.shape[0]):
        point = landmarks[i]
        x, y = int(point[0]), int(point[1])
        cv2.circle(input_image, (x, y), 1, (0, 255, 0), -1)  # Green color, filled circle

In [ ]:
# 사용자 영역
draw_landmarks("./faces/newJeans.jpg")

#### 2. 동일인 얼굴 찾기 & 얼굴 바꾸기
##### 2-1. 공통 기능 데코레이터: `recognize_and_process_faces`

  - 입력 이미지와 참조 이미지(들)를 각각 읽고, 얼굴 검출을 수행.  
  - 참조 이미지에서 검출된 얼굴들을 하나의 리스트(`ref_faces_all`)에 모음.  
  - 입력 이미지의 각 얼굴에 대해 데코레이트된 함수를 호출하면서 참조 얼굴 리스트를 인자로 전달.  
  - 처리 결과를 화면에 출력

In [ ]:
# Library 개발자 영역
from typing import Union, List

def recognize_and_process_faces(func):
    def wrapper(input_path: str, ref_paths: Union[str, List[str]]) -> None:
        
        # ref_paths가 str 타입이면, 단일 요소 리스트로 변환
        if isinstance(ref_paths, str):
            ref_paths = [ref_paths]

        # 입력 이미지 읽기
        input_image = cv2.imread(input_path)
        if input_image is None:
            print("입력 이미지를 읽을 수 없습니다. 경로를 확인하세요:", input_path)
            return

        # 입력 이미지에서 얼굴 검출
        input_faces = analyzer.get(input_image)
        if not input_faces:
            print("입력 이미지에서 얼굴을 검출하지 못했습니다.")
            return

        # 여러 참조 이미지의 얼굴들을 모두 모으기
        ref_faces_all = []
        for ref_path in ref_paths:
            ref_image = cv2.imread(ref_path)
            if ref_image is None:
                print("참조 이미지를 읽을 수 없습니다. 경로를 확인하세요:", ref_path)
                continue

            ref_faces = analyzer.get(ref_image)
            if not ref_faces:
                print("참조 이미지에서 얼굴을 검출하지 못했습니다. 경로:", ref_path)
                continue

            # 해당 참조 이미지의 얼굴들을 추가
            ref_faces_all.extend(ref_faces)

        if not ref_faces_all:
            print("모든 참조 이미지에서 얼굴을 검출하지 못했습니다.")
            return

        # 입력 이미지의 각 얼굴에 대해 데코레이트된 함수 호출 (ref_faces_all 을 전달)
        for face in input_faces:
            func(input_image, face, ref_faces_all)

        # 처리된 결과 이미지를 Notebook에서 표시 (BGR -> RGB 변환)
        output_image_rgb = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(15, 9))
        plt.imshow(output_image_rgb)
        plt.axis("off")
        plt.show()

    return wrapper

##### 1-2. 데코레이터 `recognize_and_process_faces`와 유사도 함수를 활용한 동일인 찾기

  - 참조 이미지의 첫 번째 얼굴과 입력 이미지의 각 얼굴 사이의 코사인 유사도를 계산.  
  - 유사도 값에 따라(유사도 측정값 0.4 이상상) 텍스트와 박스 색상을 변경하여 화면에 표시.

In [ ]:
# App 개발자 영역 : 특정 얼굴과의 유사도 측정해서 보여주기
from sklearn.metrics.pairwise import cosine_similarity

@recognize_and_process_faces
def show_similarity(input_image: np.ndarray, face, ref_faces):
    # 코사인 유사도 계산 (각 face의 embedding 값을 사용)
    similarity = cosine_similarity([ref_faces[0].embedding], [face.embedding])[0][0]
    
    # 얼굴의 바운딩 박스를 가져와서 정수형으로 변환 (각 모델에 따라 속성이 다를 수 있음)
    bbox = face.bbox.astype(int)  # face.bbox가 [x1, y1, x2, y2] 형태라고 가정
    
    # 유사도 값을 텍스트로 작성 (소수점 두 자리)
    label = f"Sim: {similarity:.2f}"
    
    # similarity 값에 따라 다른 색상으로 출력:
    # similarity >= 0.4이면 빨간색, 아니면 기본 색상으로 표시합니다.
    if similarity >= 0.4:
        text_color = (0, 0, 255)   # 빨간색 (BGR)
        rect_color = (0, 0, 255)
    else:
        text_color = (0, 255, 0)   # 초록색 (BGR)
        rect_color = (0, 255, 0)   #

    # 이미지에 텍스트 표시 (bounding box 위쪽)
    cv2.putText(input_image, label, (bbox[0], bbox[1]-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2)
    # 참고: 필요에 따라 bounding box 도 그릴 수 있습니다.
    cv2.rectangle(input_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), rect_color, 2)

In [ ]:
# 사용자 영역:
# - 입력 이미지: 뉴진스 이미지
# - 참조 이미지: 트럼프 이미지 (트럼프 이미지의 첫 번째 얼굴을 참조 얼굴로 사용)
show_similarity("./faces/newJeans.jpg", "./faces/hanni.jpg")

##### 1-3. 특정인을 찾은 후 얼굴 바꾸기 모델(`inswapper`) 활용 다른 얼굴로 바꾸기

  - 입력 이미지의 얼굴과 참조 이미지의 얼굴 간의 유사도를 계산하여, 특정 조건(예, 유사도 ≥ 0.4)을 만족하면 해당 얼굴을 다른 얼굴(예, 트럼프 얼굴)로 교체.
  - 교체된 결과와 유사도 정보를 이미지에 표시.

In [ ]:
# Face Swap 모델을 메모리에 로드하고 AI 기능 초기화
swapper = insightface.model_zoo.get_model( INSWAPPER_PATH )

@recognize_and_process_faces
def replace_face(input_image: np.ndarray, face, ref_faces):
    # 코사인 유사도 계산 (각 face의 embedding 값을 사용)
    similarity = cosine_similarity([ref_faces[0].embedding], [face.embedding])[0][0]
    
    # 얼굴의 바운딩 박스를 가져와서 정수형으로 변환 (각 모델에 따라 속성이 다를 수 있음)
    bbox = face.bbox.astype(int)  # face.bbox가 [x1, y1, x2, y2] 형태라고 가정
    
    # 유사도 값을 텍스트로 작성 (소수점 두 자리)
    label = f"Sim: {similarity:.2f}"
    
    # similarity >= 0.4이면 빨간색, 아니면 기본 색상으로 표시합니다.
    # similarity >= 0.4이면 얼굴 교체
    if similarity >= 0.4:
        text_color = (0, 0, 255)   # 빨간색 (BGR)
        input_image[:,:] = swapper.get(input_image, face, ref_faces[1])
    else:
        text_color = (0, 255, 0)   # 초록색 (BGR)

    # 이미지에 텍스트 표시 (bounding box 위쪽)
    cv2.putText(input_image, label, (bbox[0], bbox[1]-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2)

In [ ]:
# 사용자 영역:
# - 입력 이미지: 뉴진스 이미지
# - 참조 이미지: 하니 이미지, 트럼프 이미지 (하니를 찾아 트럼프 얼굴로 교체)
replace_face("./faces/newJeans.jpg", ["./faces/hanni.jpg", "./faces/trump.jpg"])

### Code 작성하기 – 클래스를 사용한 추상화
#### 1. FaceModifier 클래스 정의
- 얼굴 분석 및 처리 관련 기능들을 FaceModifier라는 클래스에 캡슐화하여, 
- App 개발자가 저수준의 구현 세부사항(예: OpenCV 이미지 처리, AI 모델 초기화 및 호출)을 신경 쓰지 않고, 간단한 고수준의 메소드들을 사용할 수 있도록 설계
- 앞서 정의한 데코레이터 함수들(`detect_and_process_faces`와 `recognize_and_process_faces`)을 그대로 사용

In [ ]:
class FaceModifier:

    # 클래스 상수 GREEN과 RED를 정의하여, 얼굴 박스나 텍스트 표시 시 사용
    GREEN = (0, 255, 0)
    RED = (0, 0, 255)
    
    # 생성자
    def __init__(self):
        # buffalo Face Analysis 모델을 메모리에 로드
        analyzer = FaceAnalysis(name='buffalo_l', root=BUFFALO_L_PATH) 
        analyzer.prepare(ctx_id=-1) # CPU Mode 의미

        # Face Swap 모델을 메모리에 로드
        swapper = insightface.model_zoo.get_model( INSWAPPER_PATH )
        
        self.analyzer = analyzer
        self.swapper = swapper

    # 얼굴 바운딩 박스 그리기
    def draw_face_bbox(self, input_image, face, color=(0, 255, 0)):
        bbox = face.bbox.astype(int)
        cv2.rectangle(input_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
    
    # 유사도 계산
    # - 입력 얼굴과 참조 얼굴 간의 코사인 유사도를 계산하여 반환
    def similarity(self, face, ref_face):
        return cosine_similarity([ref_face.embedding], [face.embedding])[0][0]
    
    # 얼굴 바꾸기
    def replace_face(self, input_image, face, ref_face):
        input_image[:,:] = self.swapper.get(input_image, face, ref_face)

    # 얼굴 인식 및 처리
    def detect_and_process_faces(self, func):
        return detect_and_process_faces(func)
    
    # 얼굴 비교 및 처리
    def recognize_and_process_faces(self, func):
        return recognize_and_process_faces(func)

#### 2.FaceModifier 클래스 활용 예제
##### 2-1. 클래스 인스턴스 생성

In [ ]:
fm = FaceModifier()

##### 2-2. 얼굴 박스 그리기
- 데코레이터 `@fm.detect_and_process_faces`를 사용, 지정한 이미지에서 얼굴을 자동으로 감지한 
- 이후 각 얼굴에 대해 `draw_face_bbox` 메소드가 호출되어 얼굴 영역에 박스를 그림.

In [ ]:
@fm.detect_and_process_faces
def draw_face_bbox(input_image: np.ndarray, face: Face) -> None:
    # 얼굴 영역 표시
    fm.draw_face_bbox(input_image, face)

In [ ]:
# 사용자 영역: 뉴진스 이미지의 얼굴에 박스 그리기
draw_face_bbox("./faces/newJeans.jpg")

##### 2-3. 얼굴 교체
- 데코레이터 `@fm.recognize_and_process_faces`를 사용, 입력 이미지와 참조 이미지(여러 개)를 각각 읽어 얼굴을 검출.
- 이후, 각 입력 이미지의 얼굴에 대해 참조 이미지의 첫 번째 얼굴과의 유사도를 계산. 
- 유사도가 기준(0.4) 이상이면 빨간색 박스로 표시한 후 replace_face 메소드를 통해 얼굴을 교체.

In [ ]:
@fm.recognize_and_process_faces
def replace_face(input_image: np.ndarray, face, ref_faces):
    if fm.similarity(face, ref_faces[0]) >= 0.4:
        fm.draw_face_bbox(input_image, face, FaceModifier.RED)
        fm.replace_face(input_image, face, ref_faces[1])

In [ ]:
# 사용자 영역:
# - 입력 이미지: 뉴진스 이미지
# - 참조 이미지: 하니 이미지와 트럼프 이미지 (하니 얼굴을 찾아 트럼프 얼굴로 교체)
replace_face("./faces/newJeans.jpg", ["./faces/hanni.jpg", "./faces/trump.jpg"])